Sentiment analysis of lego themed social media

steps:
1. pull in posts and comments real time
2. clean data
3. sentiment analysis of comments
4. show results as dashboard or API


In [ ]:
#To get long life access token from facebook
# Define URL
url = params['endpoint_base'] + 'oauth/access_token'

# Define Endpoint Parameters
endpointParams = dict() 
endpointParams['grant_type'] = 'fb_exchange_token'
endpointParams['client_id'] = params['client_id']
endpointParams['client_secret'] = params['client_secret']
endpointParams['fb_exchange_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
long_lived_token = json.loads(data.content)
long_lived_token


In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
import requests
import json
import datetime

insta_access_token = 'EAADn7X6UUMIBAAMmrZBbSm5mKBNX0NtLC5sDme4HhKqA8kLxP9JQ9qx0iZBRWvA0c42phOYWCZBXhb3IqX3nT9QZBdsCtNh2zRaZAaOkkFSWZBAyfGpcZCeZCO96V89qXlrnOzF09XAFfrkm9SxdoQ9V4ZB1d31G0FWon3PI7wZCpq7G4JWpCGW4sn'
page_id = '121689584245195'
instagram_account_id = '4848230760'
app_id = '255007216914626'
app_secret = '9879766d852cf6d5080d3c82e69fede3'


In [ ]:
# Define Parameters Dictionary
params = dict()
params['access_token'] = insta_access_token        
params['client_id'] = app_id                 
params['client_secret'] = app_secret     
params['graph_domain'] = 'https://graph.facebook.com'
params['graph_version'] = 'v16.0'
params['endpoint_base'] = params['graph_domain'] + '/' + params['graph_version'] + '/'
params['page_id'] = page_id                  
params['instagram_account_id'] = instagram_account_id       
params['ig_username'] = 'susohikitabi'
params['lego_id'] ='196743444'
params['lego_username'] = '@lego'

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['input_token'] = params['access_token']
endpointParams['access_token'] = params['access_token']

# Define URL
url = params['graph_domain'] + '/debug_token'

# Requests Data
data = requests.get(url, endpointParams)
access_token_data = json.loads(data.content)
access_token_data

#Check token expiry date
print("Token Expires: ", datetime.datetime.fromtimestamp(access_token_data['data']['expires_at']))

In [ ]:
# Define URL
url = params['endpoint_base'] + params['instagram_account_id'] + '/media?'

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['fields'] = 'id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username,like_count,comments_count'
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
basic_insight = json.loads(data.content)
basic_insight


In [ ]:
#create spark session
spark = SparkSession.builder.master("local[1]") \
                    .appName('LegoSentiment') \
                    .getOrCreate()

In [ ]:
#define schema here
post_comments = tp.StructType([
  tp.StructField(name= 'id',          dataType= tp.IntegerType(),  nullable= True),
  tp.StructField(name= 'label',       dataType= tp.IntegerType(),  nullable= True),
  tp.StructField(name= 'tweet',       dataType= tp.StringType(),   nullable= True)
])

In [ ]:
import tweepy

client = tweepy.Client(
    consumer_key="API / Consumer Key here",
    consumer_secret="API / Consumer Secret here",
    access_token="Access Token here",
    access_token_secret="Access Token Secret here"
)

In [ ]:
from requests_oauthlib import OAuth1Session
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'CONSUMER_KEY'='<your_consumer_key>'
# export 'CONSUMER_SECRET'='<your_consumer_secret>'

consumer_key = 'iF7AN1SReddkMvBLkN2eFTrT9'
consumer_secret = 'uWenXIZU77fBCczUlK8Oy2QkvrTBIF94gn3XFplwDf2Ijd7w9F'

# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
fields = "created_at,description"
params = {"user.fields": fields}

# Get request token
request_token_url = "https://api.twitter.com/oauth/request_token"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# # Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

response = oauth.get("https://api.twitter.com/2/users/me", params=params)

if response.status_code != 200:
    raise Exception(
        "Request returned an error: {} {}".format(response.status_code, response.text)
    )

print("Response code: {}".format(response.status_code))

json_response = response.json()

print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
pip install twint

In [1]:
import twint
import nest_asyncio

nest_asyncio.apply()


In [ ]:
# Configure
c = twint.Config()
c.Search = '#LEGO'
c.Limit = 20
twint.run.Search(c)


In [ ]:
pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [1]:
pip install instaloader

     ---------------------------------------- 60.5/60.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for instaloader: filename=instaloader-4.9.6-py3-none-any.whl size=62309 sha256=a46d38def0a13d621215a7f672005d53188828cf80d28c88162031227eac650d
  Stored in directory: c:\users\hayle\appdata\local\pip\cache\wheels\17\c8\6f\fd185ae255bb878d9f2bdb83f27a4a2b383583857167557aee
Successfully built instaloader
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
L = instaloader.Instaloader()
for post in instaloader.Hashtag.from_name(L.context, 'LEGO').get_posts():
    # post is an instance of instaloader.Post
    L.download_post(post, target='#LEGO')

#LEGO\2023-05-06_22-48-19_UTC.jpg [https://zrzutka.pl/n7srpw  #r…] json 
#LEGO\2023-05-06_22-47-51_UTC_1.jpg #LEGO\2023-05-06_22-47-51_UTC_2.jpg [Getting closer... #Lego #bric…] json 
#LEGO\2023-05-06_22-47-46_UTC.jpg [Book 1 done.  MTT lego set 76…] json 
#LEGO\2023-05-06_22-47-34_UTC_1.jpg #LEGO\2023-05-06_22-47-34_UTC_2.jpg #LEGO\2023-05-06_22-47-34_UTC_3.jpg #LEGO\2023-05-06_22-47-34_UTC_4.jpg #LEGO\2023-05-06_22-47-34_UTC_5.jpg #LEGO\2023-05-06_22-47-34_UTC_6.jpg #LEGO\2023-05-06_22-47-34_UTC_7.jpg #LEGO\2023-05-06_22-47-34_UTC_8.jpg #LEGO\2023-05-06_22-47-34_UTC_9.jpg #LEGO\2023-05-06_22-47-34_UTC_10.jpg [Kids beds made from giant Leg…] json 
#LEGO\2023-05-06_22-47-13_UTC.jpg exists [#photographer #photography #p…] updated [#inspiration #photo #lifestyl…] json 
#LEGO\2023-05-06_22-46-56_UTC_1.jpg #LEGO\2023-05-06_22-46-56_UTC_2.jpg #LEGO\2023-05-06_22-46-56_UTC_3.jpg #LEGO\2023-05-06_22-46-56_UTC_4.jpg [Brick-fest finally!!!  Haven’…] json 
#LEGO\2023-05-06_22-46-41_UTC_1.jpg #LE

In [1]:
import instaloader
import json
from instaloader import Profile, Post
import nest_asyncio

nest_asyncio.apply()

In [23]:
L = instaloader.Instaloader()
profile = Profile.from_username(L.context,'lego')
posts = profile.get_posts()
count = 0
post_list = []


for post in posts:
    if count < 4:
        L.download_post(post, 'lego')
        post_params = dict()
        comment_list = []
        post_params['Title'] = post.title
        post_params['Date'] = post.date_local
        post_params['Url'] = post.url
        post_params['Likes'] = post.likes
        post_params['Caption'] = post.caption
        post_params['Caption_Hashtags'] = post.caption_hashtags
        post_list.append(post_params)
        count=count+1
    else:
        break

lego\2023-05-05_16-04-24_UTC.jpg exists [When we received footage of a…] unchanged lego\2023-05-05_16-04-24_UTC.mp4 exists json 
lego\2023-05-05_13-00-34_UTC.jpg exists [Go on a voyage to space and d…] unchanged lego\2023-05-05_13-00-34_UTC.mp4 exists json 
lego\2023-05-05_10-00-09_UTC.jpg exists [The moon is looking lovely in…] unchanged json 
lego\2023-05-04_10-00-40_UTC_1.jpg exists lego\2023-05-04_10-00-40_UTC_2.jpg exists lego\2023-05-04_10-00-40_UTC_3.jpg exists lego\2023-05-04_10-00-40_UTC_4.jpg exists lego\2023-05-04_10-00-40_UTC_5.jpg exists lego\2023-05-04_10-00-40_UTC_6.jpg exists lego\2023-05-04_10-00-40_UTC_7.jpg exists lego\2023-05-04_10-00-40_UTC_8.jpg exists lego\2023-05-04_10-00-40_UTC_9.jpg exists lego\2023-05-04_10-00-40_UTC_10.jpg exists [The battle between good and e…] unchanged json 


In [24]:
post_list

[{'Title': '',
  'Date': datetime.datetime(2023, 5, 5, 17, 4, 24, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT Summer Time')),
  'Url': 'https://scontent-lcy1-2.cdninstagram.com/v/t51.2885-15/344867243_952040929150913_228286961473239942_n.jpg?stp=dst-jpg_e15_fr_p1080x1080&_nc_ht=scontent-lcy1-2.cdninstagram.com&_nc_cat=105&_nc_ohc=NkLKXOWAD14AX-AZtvj&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfBdEHRFQj9VLAEZWpi_ByfqLknrfpccgcCw7u-9LH09KQ&oe=64588E51&_nc_sid=8fd12b',
  'Likes': 27102,
  'Caption': 'When we received footage of an incident outside Birch Books yesterday, it wasn’t quite what we expected. We also received a box of cakes tied with a blue bow and a photograph of a rare shark. If anyone can shed any light on what’s happening here, please let us know. 🎞️🤷\n\n#LEGO #LEGOMinifigures #WesAndersonTrend',
  'Caption_Hashtags': ['lego', 'legominifigures', 'wesandersontrend']},
 {'Title': '',
  'Date': datetime.datetime(2023, 5, 5, 14, 0, 34, tzinfo=datetime.timezone(datetime.

In [25]:
import pandas as pd

In [26]:
df = pd.DataFrame(post_list)
df

,Title,Date,Url,Likes,Caption,Caption_Hashtags
0,,2023-05-05 17:04:24+01:00,https://scontent-lcy1-2.cdninstagram.com/v/t51...,27102,When we received footage of an incident outsid...,"[lego, legominifigures, wesandersontrend]"
1,,2023-05-05 14:00:34+01:00,https://scontent-lcy1-1.cdninstagram.com/v/t51...,32572,Go on a voyage to space and decorate your home...,"[lego, legoideas]"
2,None,2023-05-05 11:00:09+01:00,https://scontent-lcy1-1.cdninstagram.com/v/t51...,72986,The moon is looking lovely in red tonight! 🌕 H...,[]
3,None,2023-05-04 11:00:40+01:00,https://scontent-lcy1-1.cdninstagram.com/v/t51...,79321,The battle between good and evil is on! May th...,"[lego, starwars, afol, toyphotography]"


In [27]:
df.to_csv('C:/Users/hayle/Desktop/top_post.csv',index=False)